In [6]:
import numpy as np
import open3d as o3d
import cv2
import os
from PIL import Image
from matplotlib import pyplot as plt

In [7]:
'''
Nicolas model path, including real-world images, rendered images and their world pose parameters(intrinsic and extrinsic)
'''
ModelPath="/media/ren/EXTERNAL_USB/KITTI360_DATASET/Test_data"

print(ModelPath)

#file read
def imagepath(path):
    images_path=os.path.join(ModelPath,path)
# print(pc_path)
    realfilelist = os.listdir(images_path)
    realfilelist =sorted(realfilelist)
    return realfilelist

/media/ren/EXTERNAL_USB/KITTI360_DATASET/Test_data


In [41]:
def ORB(img):
    """
     ORB detector
    """
    orb = cv2.ORB_create(50000)
    """find keypoint, and calculate descriptor"""
    kp, des = orb.detectAndCompute(img, None)

    # plot keypoints
    # img2 = cv2.drawKeypoints(img, kp, None, color=(0, 255, 0), flags=0)

 
    # plt.figure(figsize=(10, 8), dpi=100)
    # plt.imshow(img2[:, :, ::-1])
    # plt.xticks([]), plt.yticks([])
    # plt.show()
    return kp, des



def SIFT(img):
    """
     SIFT detector
    """
    # gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # SIFT keypoint detector
    # sift instantiation 
    sift = cv2.xfeatures2d.SIFT_create()

    # sift detect and compute 
    kp, des = sift.detectAndCompute(img, None)
    
    # kp: information of keypoint, including position, scale, direction
    # des: keypoint descriptor, corresponds to a feature vector of 128 gradient information

    # image show
    # cv2.drawKeypoints(img, kp, img, (0, 255, 0))

    
    # plt.figure(figsize=(10, 8), dpi=100)
    # plt.imshow(img[:, :, ::-1])
    # plt.xticks([]), plt.yticks([])
    # plt.show()
    return kp, des


def SURF(img):
    surf = cv2.xfeatures2d.SURF_create()
    kp, des = surf.detectAndCompute(img, None)
    # cv2.drawKeypoints(img, kp, img, (0, 255, 0))

    
    # plt.figure(figsize=(10, 8), dpi=100)
    # plt.imshow(img[:, :, ::-1])
    # plt.xticks([]), plt.yticks([])
    # plt.show()
    return kp, des


In [38]:
def Brute(img1, img2, kp1, kp2, des1, des2, flag):
    """
    Brute Force matching
    :param img1: image 1
    :param img2: image 2
    :param kp1: keypoints of frame 1True
    :param kp2: keypoints of frame 2
    :param des1: descriptor of frame 1
    :param des2: descriptor of frame 2
    :return:
    """
    print("img1",len(kp1),len(kp2),len(des1),len(des2))
    # if type(des1)!=NoneType and type(des2)!=NoneType:
    if (flag == "SIFT" or flag == "SURF"):
        # SIFT
        bf = cv2.BFMatcher_create(cv2.NORM_L1, crossCheck=False)
        ms = bf.knnMatch(des1, des2, k=2)
    else:
        # ORB
        # bf = cv2.BFMatcher_create(cv2.NORM_L1, crossCheck=True)
        bf = cv2.BFMatcher_create(cv2.NORM_HAMMING, crossCheck=False)
        ms = bf.match(des1, des2)
        ms = sorted(ms, key=lambda x: x.distance)
    # print('des1',des1.shape)
    # print('des2',des2.shape)
    
    # ms = bf.match(des1, des2)
    # ms = sorted(ms, key=lambda x: x.distance)
    # img3 = cv2.drawMatches(img1, kp1, img2, kp2, ms, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    # cv2.imshow("Matches", img3)
    # cv2.waitKey(0)
    return ms

def FLANN(img1, img2, kp1, kp2, des1, des2, flag):
    """
        2. FLANN matching
        :param img1: image 1
        :param img2: image 2
        :param kp1: keypoints of frame 1
        :param kp2: keypoints of frame 2
        :param des1: descriptor of frame 1
        :param des2: descriptor of frame 2
        :return:
        """
    print("img1",len(kp1),len(kp2),len(des1),len(des2))    
    if (flag == "SIFT" or flag == "SURF"):
        # SIFT
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE,
                            trees=5)
        search_params = dict(check=100)
    else:
        # ORB
        FLANN_INDEX_LSH = 6
        index_params = dict(algorithm=FLANN_INDEX_LSH,
                            table_number=6,
                            key_size=12,
                            multi_probe_level=1)
        
        
        search_params = dict(check=100)
    # define FLANN parameter
    # print('des1',des1.shape)
    # print('des2',des2.shape)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    return matches

In [39]:
"""
optimization matching results
RANSAC(RANdom SAmple Consensus)
"""


def RANSAC(img1, img2, kp1, kp2, matches,MIN_MATCH_COUNT):
    # MIN_MATCH_COUNT = 500
    # store all the good matches as per Lowe's ratio test.
    matchType = type(matches[0])
    good = []
    successful=[]
    unsuccessful=[]
 
    # print(matchType)
    if isinstance(matches[0], cv2.DMatch):
        # Search for matching
        good = matches
    else:
        # knnMatch
        ratio_threshold=0.7
        for m, n in matches:
            if m.distance < ratio_threshold* n.distance:
                good.append(m)
    # print('number',len(good))
    
    if len(good) > MIN_MATCH_COUNT:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

        # M: 3x3 Homography matrix. last number is the error of the transformation between points on original images and target images.
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()
        # H.append(M)
        # h, w = img1.shape
        # pts = np.float32([[0, 0], [0, h - 1], [w - 1, h - 1], [w - 1, 0]]).reshape(-1, 1, 2)
        # dst = cv2.perspectiveTransform(pts, M)
        #
        # img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
    else:
        print
        "Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT)
        matchesMask = None

    draw_params = dict(matchColor=(0, 255, 0),  # draw matches in green color
                       singlePointColor=None,
                       matchesMask=matchesMask,  # draw only inliers
                       flags=2)
    good_match_len=len(good)
    # print(matchesMask)
    Suc_matching=matchesMask.count(1)
    # print(Suc_matching)
    # bad_match_len=outliermask.count(1)
    # successful.append(Suc_matching)
    Unsuc=matchesMask.count(0)
    # print(Unsuc)
    # unsuccessful.append(Unsuc)
    img3 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params)

    draw_params1 = dict(matchColor=(0, 255, 0),  # draw matches in green color
                        singlePointColor=None,
                        matchesMask=None,  # draw only inliers
                        flags=2)
    # print('matched',kp1.size,kp2.size)
    img33 = cv2.drawMatches(img1, kp1, img2, kp2, good, None, **draw_params1)
    # print("good len",len(good))
    # print("kp1",kp1[0].distance)

    # cv2.imshow("before", img33)
    # cv2.imshow("now", img3)
    # cv2.imwrite('matching testdata00_0000.jpg',img3)
    # cv2.waitKey(20)
    return good_match_len,Suc_matching,Unsuc,M


In [ ]:
# read extrinsic and intrinsic

parameters=o3d.io.read_pinhole_camera_parameters(os.path.join(ModelPath,"world_poses","world_pose_00008.json"))
intrinsic=parameters.intrinsic.intrinsic_matrix
extrinsic=parameters.extrinsic

print(intrinsic,extrinsic)


[[3.44823034e+03 0.00000000e+00 2.11650000e+03]
 [0.00000000e+00 3.43126050e+03 1.40800000e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]] [[ 9.90164574e-01  1.39503036e-01  1.06310234e-02 -2.27301571e+01]
 [-2.89960020e-02  2.78955743e-01 -9.59866098e-01  4.58700839e+00]
 [-1.36869819e-01  9.50117149e-01  2.80257125e-01  1.17073411e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [42]:
'''
Ground Truth Value, make sure that frames are from the same sequence
'''

def GroundTruthPose(framename1,framename2):
    param_frame1=o3d.io.read_pinhole_camera_parameters(os.path.join(ModelPath,"world_poses","world_pose_00%d.json"framename1))
    intri_1=param_frame1.intrinsic.intrinsic_matrix
    extri_1=param_frame1.extrinsic


SyntaxError: invalid syntax (<ipython-input-42-4f8ce22778c2>, line 6)